In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np
from torch.autograd import Variable

import re

In [19]:
df = pd.read_csv('poet.tang.csv')
df_dict = pd.read_csv('tang.dict.csv')

In [114]:
wordlist = []
for s in df.poets:
    '''
    s = s.replace("'", "").replace(", ", "").replace("[", "").replace("]", "")
    s = s.replace("，", "").replace("。", "").replace("《"，)
    '''
    s = re.sub("[\[\'\,\ \[\]，。\]《》！（）：；？Ｂ［］｛｝\+\-\/0123456789\=\\a-z\{\|\}·…□○●、〈〉《》「」『』〖〗]", "", s)
    wordlist.extend(list(s))

wordlist.sort()

wordcnt = {}    
word2id = {}
id2word = {}

i = 1
for c in wordlist:
    if c not in word2id:
        word2id[c] = i
        id2word[i] = c
        wordcnt[c] = 1
        i += 1
    else:
        wordcnt[c] += 1
        
wordlist = list(set(wordlist))
wordlist.sort()

In [110]:
wordlist.sort()
wordlist[:50]
wordcnt['丶']
df.iloc[979]

Unnamed: 0                                                 1024
authors                                                      李白
poets         ['幽州胡馬客，綠眼虎皮冠。', '笑拂兩隻箭，萬人不可干。', '彎弓若轉月，白雁落雲端。...
strains       ['平平平仄仄，仄仄仄平○。', '仄仄仄仄仄，仄平仄仄平。', '平平仄仄仄，仄仄仄平平。...
titles                                              橫吹曲辭 幽州胡馬客歌
lenth                                                       141
Name: 979, dtype: object

In [100]:
df[df.lenth>100]

,Unnamed: 0,authors,poets,strains,titles,lenth
11,11,太宗皇帝,"['執契靜三邊，持衡臨萬姓。', '玉彩輝關燭，金華流日鏡。', '無爲宇宙清，有美璇璣正。...","['仄仄仄○平，平平○仄仄。', '仄仄平平仄，平平平仄仄。', '平平仄仄平，仄仄平平○。...",執契靜三邊,200
13,13,太宗皇帝,"['壽丘惟舊跡，酆邑乃前基。', '粵予承累聖，懸弧亦在茲。', '弱齡逢運改，提劒鬱匡時。...","['仄平平仄仄，平仄仄平平。', '仄仄平仄仄，平平仄仄平。', '仄平平仄仄，平仄仄平平。...",幸武功慶善宮,106
24,24,太宗皇帝,"['未央初壯漢，阿房昔侈秦。', '在危猶騁麗，居奢遂役人。', '豈如家四海，日宇罄朝倫。...","['仄平平仄仄，平平仄仄平。', '仄平○仄仄，平平仄仄平。', '仄○平仄仄，仄仄仄平平。...",登三臺言志,110
40,40,太宗皇帝,"['碧昏朝合霧，丹卷暝韜霞。', '結葉繁雲色，凝璚徧雪華。', '光樓皎若粉，映幕集疑沙。...","['仄平平仄仄，平仄○平平。', '仄仄平平仄，○○仄仄平。', '平平仄仄仄，仄仄仄平平。...",喜雪,110
43,43,太宗皇帝,"['二儀初創象，三才乃分位。', '非惟樹司牧，固亦垂文字。', '緜代更膺期，芳圖無輟記。...","['仄平平仄仄，○平仄○仄。', '平平仄○仄，仄仄平平仄。', '平仄○平○，平平平仄仄。...",詠司馬彪續漢志,220
100,100,高宗皇帝,"['龍樓光曙景，魯館啓朝扉。', '豔日濃妝影，低星降婺輝。', '玉庭浮瑞色，銀牓藻祥徽。...","['平平平仄仄，仄仄仄平平。', '仄仄平平仄，平平○仄平。', '仄○平仄仄，平仄仄平平。...",太子納妃太平公主出降,120
112,113,中宗皇帝,"['潤色鴻業寄賢才，叨居右弼媿鹽梅。', '運籌帷幄荷時來，職掌圖籍濫蓬萊。', '兩司謬忝...","['仄仄平仄仄平平，平平仄仄仄仄平。', '仄平平仄○平平，仄仄平仄仄平平。', '仄○仄仄...",十月誕辰內殿宴羣臣效柏梁體聯句,112
116,117,明皇帝,"['緬想封唐處，實惟建國初。', '俯察伊晉野，仰觀乃參虛。', '井邑龍斯躍，城池鳳翔餘。...","['仄仄○平仄，仄平仄仄平。', '仄仄平仄仄，仄○仄○平。', '仄仄平平仄，平平仄平平。...",過晉陽宮,120
127,128,明皇帝,"['洛陽芳樹映天津，灞岸垂楊窣地新。', '直爲經過行處樂，不知虛度兩京春。', '去年餘閏...","['仄平平仄仄平平，仄仄平平仄仄平。', '仄平平○○仄仄，仄平平仄仄平平。', '仄平平仄...",初入秦川路逢寒食,140
128,129,明皇帝,"['暇景屬三春，高臺聊四望。', '目極千里際，山川一何壯。', '太華見重巖，終南分疊嶂。...","['仄仄仄○平，平平平仄仄。', '仄仄平仄仄，平平仄平仄。', '仄平仄○平，平平○仄仄。...",春臺望,176


In [115]:
class PoetryModel(nn.Module):
    
    def __init__(self, vocab, hidden_size, n_cat, bs=1, nl=2):
        super().__init__()
        self.hidden_size = hidden_size
        self.bs = bs
        self.nl = nl
        self.e = nn.Embedding(n_vocab, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, nl)
        self.fc2 = nn.Linear(hidden_size, n_cat)
        self.softmax = nn.LogSoftmax(dim=-1)
        
    def forward(self, inp):
        bs = inp.size(1)
        # if bs != self.bs:
        #     self.bs = bs
            
        # print(inp.size())
        e_out = self.e(inp)
        
        # print('e_out ', e_out.size())
        h0 = c0 = Variable(e_out.data.new(*(self.nl, bs, self.hidden_size)).zero_())
        # print('h0 c0', h0.size(), c0.size())
        rnn_o,_ = self.lstm(e_out, (h0, c0)) 
        # print('rnn_o', rnn_o.size())
        rnn_o = rnn_o[-1]
        # print('rnn_o', rnn_o.size())
        fc = F.dropout(self.fc2(rnn_o), p=0.8)
        # print('fc', fc.size())
        out = self.softmax(fc)
        # print('out', out.size())
        return out

n_vocab = 9015  # len(wordlist)
n_hidden = 256
n_cat = n_vocab
bs = 128

model = PoetryModel(n_vocab, n_hidden, n_cat, bs=32)
model = model.cuda()

In [116]:
model.load_state_dict(torch.load('./model.pt'))

In [118]:
def test(c):
    s = ''
    if len(c) != 1:
        print("仅需一个字")
    if c not in word2id:
        print("请输入繁体")
    else:
        while len(s) < 20:
            input = word2id[c]
            input = torch.from_numpy(np.array([input])).long().cuda()
            input = input.view(1, 1)
            # print(input)
            # print(input.size())
            output = model(input)
            idx = output.argmax(dim=1)
            # print(output, 'size:', output.size())
            c = id2word[idx.item()]
            s += c
            
            
    return s

test('國')

'靘馹廋廋廋廋廋廋廋廋廋廋廋廋廋廋廋廋廋廋'

tensor([8946], dtype=torch.int32)
torch.Size([1])


['']

In [78]:
import re
s = "Example String]"
replaced = re.sub('[ES]', 'a', s)
print(replaced)

axample atring
